## Estimating lake area from satellite images


*18th - 19th October, 2021 - Léonard Dupont*

*Station d'Écologie Théorique et Expérimentale du CNRS, 09200 Moulis*

Here, we use both CIElab and RGB channels to try and estimate the area of mountain lakes. This pipeline combines:

    • k-means clustering of pixel values : to discriminate water from other surfaces. This process is imprecise and requires further mining. We optimise K to find the best possible number of clusters (might not work well with poor satellite data). 
    
    • contour detection using OpenCV : we binarise the satellite view based on previous clusters and detect the  emerging shapes. These are further sorted based on area, convexity, inertia to output a small number of relatively big, lake-alike shapes.
    
    • the cross comparison of shape masks and cluster values allows us to eliminate big shapes that encompass large portions of the image. Theoretically, only the lake is big and unites common cluster values (hard to think about without drawing). We simply use the std of cluster values within the shape to find the most compact one (hopefully: our lake).
    
    • using the scale on the screenshot, TADAAAAAM! 


In [1]:
from getpaths import pathupdate ; pathupdate()

In [2]:
import matplotlib.pyplot as plt 
import numpy as np
from math import *
import os as os 
import glob as glob
import cv2 as cv2
from tqdm import tqdm
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from kneed import KneeLocator
from PIL import Image

We previously transformed satellite views in RGB to the CIElab colourspace (see R script - to be added here).

### VERSION 1

In [32]:
Zones = ['Ayous','Bassies','Neouvielle','Camporells']

all_lake_areas = []
all_lake_names = []

for z in range(len(Zones)):
    #first we go to the directory we need
    if z==0:
        gothere = './' + Zones[z] 
    else:
        gothere = '../' + Zones[z]
    os.chdir(gothere)
    
    #list lakes to analyse
    lakes = glob.glob('*png')
    lake_areas = []
    lake_names = []
    
    for l in range(len(lakes)):
        lkim = cv2.imread(lakes[l])
        
        os.chdir('./CIElab_images')
        ciefile = glob.glob(lakes[l].split('.')[0]+'*')
        cieim = cv2.imread(ciefile[0])
        os.chdir('../')
        
        #plot the image
        #fig,ax = plt.subplots(1,2,figsize=(12,7))
        #ax = ax.ravel()
        #ax[0].imshow(lkim)
        #ax[0].set_title('RGB_image')
        #ax[1].imshow(cieim)
        #ax[1].set_title('CIElab_image')
        #plt.tight_layout()
        
        x = np.shape(cieim)[0]
        y = np.shape(cieim)[1]

        #creating the clustering matrix - centered and reduced values ? 
        colmat = np.zeros((x,y,3))
        colmat[:,:,0] = (cieim[:,:,0]-np.mean(cieim[:,:,0]))/np.std(cieim[:,:,0])
        colmat[:,:,1] = (cieim[:,:,2]-np.mean(cieim[:,:,2]))/np.std(cieim[:,:,2])
        colmat[:,:,2] = (lkim[:,:,2]-np.mean(lkim[:,:,2]))/np.std(lkim[:,:,2])
        
        # get a visual idea of point distribution in the 3D space
        #fig = plt.figure()
        #ax = fig.add_subplot(111,projection='3d')
        #ax.scatter(colmat[:,:,0],colmat[:,:,1],colmat[:,:,2])
        #plt.show()
        
        
        colmat = np.reshape(colmat,(x*y,3))
        
        
        if not(os.path.isfile('./clustered_im/gr_clusters_' + lakes[l].split('.')[0] + '.png')):
            #let's find the optimal number of clusters using kmeans++ 
            Ninit = 10
            Nmin = 2
            Nmax = 10

            dinert = []
            print('Processing lake ' + lakes[l].split('.')[0] + '.')
            for Nc in tqdm(range(Nmin,Nmax+1)):
                kk = KMeans(n_clusters=Nc, n_init=Ninit, init='k-means++').fit(colmat)
                res = kk.predict(colmat)
                dinert += [kk.inertia_]

            kn = KneeLocator(np.linspace(Nmin,Nc,Nc-1), dinert , curve = 'convex', direction = 'decreasing')
            #print('Optimal number of cluster is : ' + str(int(kn.knee)))
            #and the we get the final result
            kk = KMeans(n_clusters=int(kn.knee), n_init=10, init='k-means++').fit(colmat)
            res = kk.predict(colmat)
            res = np.reshape(res,(x,y))

            #we can still take a look at the optimisation curve
            #fig,ax = plt.subplots(1,3, figsize=(15,9))
            #ax = ax.ravel()
            #ax[0].plot(np.linspace(2,Nc,Nc-1), dinert)
            #ax[0].vlines(kn.knee, np.min(dinert), np.max(dinert), linestyles = 'dashed')
            #ax[0].set_xlabel('Number of clusters')
            #ax[0].set_ylabel('SSD')
            #ax[1].imshow(res)
            #ax[1].set_title('Clustering result with k='+str(int(kn.knee)))
            #ax[2].imshow(lkim)
            #plt.show()
            
            if not(os.path.isdir('clustered_im')):
                os.mkdir('clustered_im')
            #saving the coloured clusters
            plt.figure()
            plt.imshow(res)
            plt.savefig('./clustered_im/km_res_'+lakes[l].split('.')[0]+'.png', dpi=300)
            plt.close()
        
            #convert results to cv2-compatible image
            res = kk.predict(colmat)
            res = np.reshape(res,(x,y))
            res = np.uint32(res)
            rescaled = (255.0 / res.max() * (res - res.min())).astype(np.uint8)

            new_p = Image.fromarray(rescaled)
            new_p = new_p.convert("L")
            
            new_p.save('./clustered_im/gr_clusters_' + lakes[l].split('.')[0] + '.png')
        
        #now let's find the contours
        clslk = cv2.imread('./clustered_im/gr_clusters_' + lakes[l].split('.')[0] + '.png', cv2.IMREAD_GRAYSCALE)
        
        
        # we prepare lists to store areas, contour indices and all the produced contours
        ct_all_areas = []
        ct_all_idx = []
        all_ctrs = []
        for k in range(int(kn.knee)):
            res2 = (res==k) #boolean mask 
            res2 = np.asarray(res2, dtype='uint8') #cv-compatible mask
            res2 = 1 - res2 #inverting it : contours are detected based on black islands
            #plt.figure(figsize=(20,10))
            #plt.imshow(res2)

            # we use OpenCV to detect contours, regardless of their characteristics at first
            ct,hier = cv2.findContours(res2, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
            ct_areas = []
            ct_idx = []
            Ath = 1e4 #THRESHOLD AREA
            for i in range(len(ct)):
                A = cv2.contourArea(ct[i])
                if A > Ath : 
                    ct_areas.append(A)
                    ct_idx.append(i)
            ct_all_areas.append(ct_areas)
            ct_all_idx.append(ct_idx)
            all_ctrs.append(ct)
    
        all_stds = []
        for k in range(int(kn.knee)):
            stds = []
            n = len(ct_all_areas[k])
            for i in range(n):
                idx = ct_all_idx[k][i]
                ct = all_ctrs[k]
                mask = np.zeros_like(clslk)
                cv2.drawContours(mask, ct, idx, color=255, thickness=-1)
                poi = np.where(mask==255) #these are our points
                Npoi = np.shape(poi)[1]
                mask_cl_vals = np.zeros(Npoi)
                for j in tqdm(range(Npoi)):
                    mask_cl_vals[j] = res[poi[0][j],poi[1][j]]
                stds.append(np.std(mask_cl_vals))
            all_stds.append(stds)  
            

        kmin=0
        imin=0
        for k in range(int(kn.knee)):
            for i in range(len(ct_all_areas[k])):
                if all_stds[k][i] < all_stds[kmin][imin]:
                    kmin = k
                    imin = i
                    

        #print('The lake surface is (in px): ' + str(ct_all_areas[kmin][imin]) +'.')
        
        #saving the mask figure as well
        idx = ct_all_idx[kmin][imin]
        ct = all_ctrs[kmin]
        mask = np.zeros_like(clslk)
        cv2.drawContours(mask, ct, idx, color=255, thickness=-1)
        
        #saving mask
        plt.figure()
        plt.imshow(mask, cmap='gray')
        plt.savefig('./clustered_im/lake_mask_'+lakes[l].split('.')[0]+'.png', dpi=300)
        plt.close()
        
        
        #storing the data
        lake_areas.append(ct_all_areas[kmin][imin])
        lake_names.append(lakes[l])
    
    all_lake_areas.append(lake_areas)
    all_lake_names.append(lake_names)

    
#writing results in a csv
fields = ['Zone', 'Lake', 'Surface (px)']
with open('../clustering_results.csv','w') as f:
    write = csv.writer(f, delimiter=',')
    write.writerow(fields)
    for z in range(len(Zones)):
        for l in range(len(all_lake_names[z])):
            rr = [all_lake_zones[z][l], all_lake_names[z][l].split('.')[0], str(all_lake_areas[z][l])]
            write.writerow(list(rr))
    f.close()

        
        
        

100%|██████████| 45004/45004 [00:00<00:00, 957305.07it/s]


Processing lake Roumassot.


100%|██████████| 23237/23237 [00:00<00:00, 921819.39it/s]


Processing lake Paradis.


100%|██████████| 138332/138332 [00:00<00:00, 994135.76it/s]


...

100%|██████████| 759136/759136 [00:00<00:00, 802766.27it/s]


Processing lake Long.


100%|██████████| 56903/56903 [00:00<00:00, 743152.04it/s]


Processing lake ras_de_la_sal_aval.


100%|██████████| 20062/20062 [00:00<00:00, 647516.98it/s]


# Comments after complete run of version 1

### General remarks

• Calculation time was relatively important (1h30 for 58 lakes, so around 1min33 per lake). However, the number of clusters was consistently below 8. Neglecting K > 7 could be an interesting option, all the more since these high Ks require most of the computation time. 

• Success rate was 56.9% (error rate 43.1%) over the 58 samples. Among the errors, a great majority turned out as a surprise and should be easily detected after a few changes (see below).

• Most errors were due to :

        - shadows from the trees and mountains around.
        - irregularities in bathymetry, translating into shades of blue/green and/or uneven vegetation coverage (herbs, algal patches).
        - solid patches of rocks or forest competing in the np.min(std) department with our lake. 

### Possible updates

In this regard, we list a few Verbesserungen that should not be too costly to implement:

1 • If 2 contours have a low std, use both *position within the image* (centered groups should be privileged) and perhaps *mean (RG)B color of the patch* (lakes should have a higher B value). The second prop. might not work well when lakes are green. Use Campo_Peric and Campo_Inf to work on this. 

2 • From what I saw, the **b** channel from l\*a\*b  is not contributing well to lake detection. The **a** one might be more interesting, especially when it comes to bathymetry problems. Use Campo_Inf and Campo_Sup to verifiy this. 

3 • For bathymetry issues, smoothing of the **a** channel could also be of use. However, this could lead to the smooting of the banks in some cases... 

4 • Trees and shadows around the banks are often detected as lake... Only computer vision could solve this. 

## VERSION 2

In [269]:
Zones = ['Ayous','Bassies','Neouvielle','Camporells']

all_lake_areas = []
all_lake_names = []

for z in range(len(Zones)):
    print('Analysing lakes from '+str(Zones[z]))
    #first we go to the directory we need
    if z==0:
        gothere = './' + Zones[z] 
    else:
        gothere = '../' + Zones[z]
    os.chdir(gothere)
    
    #list lakes to analyse
    lakes = glob.glob('*png')
    lake_areas = []
    lake_names = []
    
    for l in tqdm(range(len(lakes))):
        print('Processing lake ' + lakes[l].split('.')[0] + '.')
        
        lkim = cv2.imread(lakes[l])
        
        os.chdir('./CIElab_images')
        ciefile = glob.glob(lakes[l].split('.')[0]+'*')
        cieim = cv2.imread(ciefile[0])
        os.chdir('../')
        
        #plot the image
        #fig,ax = plt.subplots(1,2,figsize=(12,7))
        #ax = ax.ravel()
        #ax[0].imshow(lkim)
        #ax[0].set_title('RGB_image')
        #ax[1].imshow(cieim)
        #ax[1].set_title('CIElab_image')
        #plt.tight_layout()
        
        x = np.shape(cieim)[0]
        y = np.shape(cieim)[1]

        #creating the clustering matrix - centered and reduced values ? 
        colmat = np.zeros((x,y,3))
        colmat[:,:,0] = (cieim[:,:,0]-np.mean(cieim[:,:,0]))/np.std(cieim[:,:,0])
        colmat[:,:,1] = (cieim[:,:,1]-np.mean(cieim[:,:,1]))/np.std(cieim[:,:,1])
        colmat[:,:,2] = (lkim[:,:,2]-np.mean(lkim[:,:,2]))/np.std(lkim[:,:,2])
        
        # get a visual idea of point distribution in the 3D space
        #fig = plt.figure()
        #ax = fig.add_subplot(111,projection='3d')
        #ax.scatter(colmat[:,:,0],colmat[:,:,1],colmat[:,:,2])
        #plt.show()
        
        
        colmat = np.reshape(colmat,(x*y,3))
        
        
        if not(os.path.isfile('./clustered_im/gr_clusters_' + lakes[l].split('.')[0] + '.png')):
            #let's find the optimal number of clusters using kmeans++ 
            Ninit = 10
            Nmin = 2
            Nmax = 7

            dinert = []
            
            for Nc in range(Nmin,Nmax+1):
                kk = KMeans(n_clusters=Nc, n_init=Ninit, init='k-means++').fit(colmat)
                res = kk.predict(colmat)
                dinert += [kk.inertia_]

            kn = KneeLocator(np.linspace(Nmin,Nc,Nc-1), dinert , curve = 'convex', direction = 'decreasing')
            #print('Optimal number of cluster is : ' + str(int(kn.knee)))
            #and the we get the final result
            kk = KMeans(n_clusters=int(kn.knee), n_init=10, init='k-means++').fit(colmat)
            res = kk.predict(colmat)
            res = np.reshape(res,(x,y))

            #we can still take a look at the optimisation curve
            #fig,ax = plt.subplots(1,3, figsize=(15,9))
            #ax = ax.ravel()
            #ax[0].plot(np.linspace(2,Nc,Nc-1), dinert)
            #ax[0].vlines(kn.knee, np.min(dinert), np.max(dinert), linestyles = 'dashed')
            #ax[0].set_xlabel('Number of clusters')
            #ax[0].set_ylabel('SSD')
            #ax[1].imshow(res)
            #ax[1].set_title('Clustering result with k='+str(int(kn.knee)))
            #ax[2].imshow(lkim)
            #plt.show()
            
            if not(os.path.isdir('clustered_im')):
                os.mkdir('clustered_im')
            #saving the coloured clusters
            plt.figure()
            plt.imshow(res)
            plt.savefig('./clustered_im/km_res_'+lakes[l].split('.')[0]+'.png', dpi=300)
            plt.close()
        
            #convert results to cv2-compatible image
            res = kk.predict(colmat)
            res = np.reshape(res,(x,y))
            res = np.uint32(res)
            rescaled = (255.0 / res.max() * (res - res.min())).astype(np.uint8)

            new_p = Image.fromarray(rescaled)
            new_p = new_p.convert("L")
            
            new_p.save('./clustered_im/gr_clusters_' + lakes[l].split('.')[0] + '.png')
        
        #now let's find the contours
        clslk = cv2.imread('./clustered_im/gr_clusters_' + lakes[l].split('.')[0] + '.png', cv2.IMREAD_GRAYSCALE)
        
        
        # we prepare lists to store areas, contour indices and all the produced contours
        ct_all_areas = []
        ct_all_idx = []
        all_ctrs = []      
        
        #building the bordermask
        bordermask = np.zeros((x,y))
        xfrac = 6
        yfrac = 7
        for r in range(x):
            for c in range(y):
                if (0 <= r <= x/xfrac) or ((xfrac-1)*x/xfrac <= r <= x) or (0 <= c <= y/yfrac) or ((yfrac-1)*y/yfrac <= c <= y):
                    bordermask[r,c] = 1 
        
        #now we identify and sort the surfaces
        for k in range(int(kn.knee)):
            res2 = (res==k) #boolean mask 
            res2 = np.asarray(res2, dtype='uint8') #cv-compatible mask
            #res2 = 1 - res2 #inverting it : contours are detected based on black islands
            #plt.figure(figsize=(20,10))
            #plt.imshow(res2)

            # we use OpenCV to detect contours, regardless of their characteristics at first
            ct,hier = cv2.findContours(res2, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
            ct_areas = []
            ct_idx = []
            Ath = 1e4 #THRESHOLD AREA
            for i in range(len(ct)):
                A = cv2.contourArea(ct[i])
                if A > Ath : 
                    Mx = 0
                    My = 0
                    for pt in range(np.shape(ct[i])[0]): #we calculate the barycentre of the surface
                        Mx += ct[i][pt][0][0]
                        My += ct[i][pt][0][1]
                    Mx = round(Mx/np.shape(ct[i])[0])
                    My = round(My/np.shape(ct[i])[0])
                    if My < x and Mx < y:
                        if bordermask[My,Mx]==0 : #reversed, but normal (I checked, contours does the reverse)
                            ct_areas.append(A)
                            ct_idx.append(i)
            if len(ct_areas)!=0: 
                ct_all_areas.append(ct_areas)
                ct_all_idx.append(ct_idx)
                all_ctrs.append(ct)
    
        all_stds = []
        for k in range(len(ct_all_areas)):
            stds = []
            n = len(ct_all_areas[k])
            for i in range(n):
                idx = ct_all_idx[k][i]
                ct = all_ctrs[k]
                mask = np.zeros_like(clslk)
                cv2.drawContours(mask, ct, idx, color=255, thickness=-1)
                poi = np.where(mask==255) #these are our points
                Npoi = np.shape(poi)[1]
                mask_cl_vals = np.zeros((int(kn.knee)))
                for j in range(Npoi):
                    cat = res[poi[0][j],poi[1][j]]
                    mask_cl_vals[cat] += 1
                mask_cl_vals = mask_cl_vals/Npoi
                id_cl = np.where(mask_cl_vals == np.max(mask_cl_vals))
                mask_cl_vals = np.delete(mask_cl_vals, id_cl)
                stds.append(sum(mask_cl_vals))
            all_stds.append(stds)
            

        kmin=0
        imin=0
        for k in range(len(ct_all_areas)):
            for i in range(len(ct_all_areas[k])):
                if all_stds[k][i] < all_stds[kmin][imin]:
                    kmin = k
                    imin = i
                    

        #print('The lake surface is (in px): ' + str(ct_all_areas[kmin][imin]) +'.')
        
        #saving the mask figure as well
        idx = ct_all_idx[kmin][imin]
        ct = all_ctrs[kmin]
        mask = np.zeros_like(clslk)
        cv2.drawContours(mask, ct, idx, color=255, thickness=-1)
        
        #saving mask
        plt.figure()
        plt.imshow(mask, cmap='gray')
        plt.savefig('./clustered_im/lake_mask_'+lakes[l].split('.')[0]+'.png', dpi=300)
        plt.close()
        
        
        #storing the data
        lake_areas.append(ct_all_areas[kmin][imin])
        lake_names.append(lakes[l])
    
    all_lake_areas.append(lake_areas)
    all_lake_names.append(lake_names)

    
#writing results in a csv
fields = ['Zone', 'Lake', 'Surface (px)']
with open('../clustering_results.csv','w') as f:
    write = csv.writer(f, delimiter=',')
    write.writerow(fields)
    for z in range(len(Zones)):
        for l in range(len(all_lake_names[z])):
            rr = [all_lake_zones[z][l], all_lake_names[z][l].split('.')[0], str(all_lake_areas[z][l])]
            write.writerow(list(rr))
    f.close()
    
os.chdir('../')

Analysing lakes from Ayous


  0%|          | 0/9 [00:00<?, ?it/s]

Processing lake Bersau.


 11%|█         | 1/9 [00:22<03:00, 22.56s/it]

Processing lake Roumassot.


...

 89%|████████▉ | 17/19 [12:29<01:32, 46.01s/it]

Processing lake Long.


 95%|█████████▍| 18/19 [13:23<00:48, 48.38s/it]

Processing lake ras_de_la_sal_aval.


100%|██████████| 19/19 [14:01<00:00, 44.30s/it]
